In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate>=0.21.0
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00


In [55]:
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [56]:
import pandas as pd
data=pd.read_excel('/content/ArzEn-MultiGenre-version-1-cleaned.xlsx')
data.head(5)

,Unnamed: 0,egyption_Text,english_Text,category,sub_category
0,0,الأمير الصغير,The little prince,Novels,the-little-prince
1,1,أنطوان دي سانت إكزوبيري,Antoine De Saint-Exuper,Novels,the-little-prince
2,2,الفصل الأول,Chapter 1,Novels,the-little-prince
3,3,في مره، لما كان عندي ست سنين، شفت صوره روعه، ف...,Once when I was six years old I saw a magnific...,Novels,the-little-prince
4,4,وكانت عباره عن تعبان من نوع البوا بيبلع فهد.,It was a picture of a boa constrictor in the a...,Novels,the-little-prince


In [57]:
data['category'].value_counts()

Subtitles      8448
Novels         5196
Song-Lyrics    3065
Name: category, dtype: int64

In [58]:
df = data.iloc[:, 1:]

train_dataset = df.iloc[:, :-2]
train_dataset

,egyption_Text,english_Text
0,الأمير الصغير,The little prince
1,أنطوان دي سانت إكزوبيري,Antoine De Saint-Exuper
2,الفصل الأول,Chapter 1
3,في مره، لما كان عندي ست سنين، شفت صوره روعه، ف...,Once when I was six years old I saw a magnific...
4,وكانت عباره عن تعبان من نوع البوا بيبلع فهد.,It was a picture of a boa constrictor in the a...
...,...,...
16704,قانون رفعت رقم واحد بعد التعديل,Refaat’s first law after an amendment:
16705,بالتأكيد هناك ما يسمى ما وراء الطبيعة,The paranormal surely exists
16706,بس الله الرحمن الرحيم! أنت مين؟,You scared me! Who are you?
16707,أنا في المكتب. رفعت مش هنا,"I’m at Refaat’s office, but he’s not here."


In [59]:
train_df = pd.DataFrame(train_dataset, columns=['english_Text', 'egyption_Text'])

In [60]:
def preprocess_data(dataframe, max_seq_length):
    processed_examples = []

    for index, row in dataframe.iterrows():
        source_text, target_text = row['english_Text'], row['egyption_Text']
        source_encoding = tokenizer(source_text, padding="max_length", truncation=True, max_length=max_seq_length, return_tensors="pt")
        target_encoding = tokenizer(target_text, padding="max_length", truncation=True, max_length=max_seq_length, return_tensors="pt")

        processed_examples.append({"input_ids": source_encoding.input_ids.flatten(),
                                   "attention_mask": source_encoding.attention_mask.flatten(),
                                   "labels": target_encoding.input_ids.flatten()})

    return processed_examples

In [62]:
max_seq_length = 128
train_dataset = preprocess_data(train_df, max_seq_length)

In [63]:
training_args = TrainingArguments(
    output_dir="./mt5_fine_tuned",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=1000,
    evaluation_strategy="steps",
    eval_steps=500,
    save_total_limit=3,
)

In [64]:
from sklearn.model_selection import train_test_split
train_data, eval_data = train_test_split(train_dataset, test_size=0.2, random_state=42)

In [65]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [66]:
# 4. Fine-tune the Model
trainer.train()


Step,Training Loss,Validation Loss
500,7.766300,4.043127
1000,1.362300,0.960699
1500,1.158600,0.934163
2000,1.209700,1.763405
2500,1.106100,1.135492
3000,0.937500,0.921748
3500,0.937300,0.850303
4000,0.910000,0.826481
4500,0.916900,0.818331
5000,0.874700,0.816467


TrainOutput(global_step=5013, training_loss=3.317518281812989, metrics={'train_runtime': 3001.8299, 'train_samples_per_second': 13.359, 'train_steps_per_second': 1.67, 'total_flos': 5300847838494720.0, 'train_loss': 3.317518281812989, 'epoch': 3.0})

In [67]:
# 5. Evaluate the Model (if you have a validation dataset)
# validation_dataset = dataset['validation'].map(preprocess_function, batched=True)
# trainer.evaluate(validation_dataset)


In [68]:
# 6. Save the Model
trainer.save_model("./fine_tuned_model")

In [69]:
# Optionally, you can also save the tokenizer
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/spiece.model',
 './fine_tuned_model/added_tokens.json')